In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import cx_Oracle
import getpass
import requests

#######Database connection###########
def con_database():
    #dsn_tns = cx_Oracle.makedsn('bandb-at1.test.avc.edu', '1521', service_name='avcprod.avc.edu') # if needed, place an 'r' before any parameter in order to address special characters such as '\'.
    dsn_tns = cx_Oracle.makedsn('bandb-at1.test.avc.edu', '1521', 'avcprep')
    while True:
        try:
            _connection = cx_Oracle.connect(user=getpass.getpass("Username: "), password=getpass.getpass("Password: "),
                                           dsn=dsn_tns, #'AVCPROD', #dsn_tns, #'AVCPREP',
                                           encoding="UTF-8")
            #print(connection)
            return _connection
        except Exception as e:
            print(e)

def close_db_cur():
    cur.close()

def close_db_connection():
    connection.close()

def exit_db():
    close_db_cur()
    close_db_connection()
####### END Database connection###########

In [ ]:
connection = con_database()
cur = connection.cursor()
print(connection.version)

In [ ]:
def define_table(table):
    _query = f"""select * from {table}"""
    for column in cur.description:
        print(column)
    #return app_data

define_table('FORAQUS')

In [ ]:
def db_query(cur,_select,_from,_where,_order_by,_group_by,_max_records_returned):
    _select = f"""select  {_select}"""
    _from = f"""from {_from}"""
    _where = (f"""where {_where}""" if _where else '')
    _order_by = (f"""order by {_order_by}""" if _order_by else '')
    _group_by = (f"""group by {_group_by}"""  if _group_by else '')
    _max_records_returned = (f"""fetch next {_max_records_returned} rows only""" if _max_records_returned else '')
    _query = f"""{_select} {_from} {_where} {_group_by} {_order_by} {_max_records_returned}"""

    print(_query)
    cur.execute(_query)
    app_data = cur.fetchall()



    return app_data


In [ ]:
select = f"""FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE"""
from_ = f"""FORAQUS"""
where = f""""""
order_by = f"""FORAQUS_QUEUE_ID"""
group_by = f""""""
max_records_returned = '' #10 #f"""10"""
x = db_query(cur,select,from_,where,order_by,group_by,max_records_returned)
for y in x:
    print(y)

In [ ]:
_select = f"""select FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE"""
_from = f"""from FORAQUS"""
_where = f""""""

x = db_query(cur,_select,_from,_where,_order_by)
for y in x:
    print(y)

In [ ]:
_select = f"""distinct FORAQUS_QUEUE_ID, FORAQUS_USER_ID_APPR,
listagg(distinct FORAQRC_ORGN_CODE,', '), listagg(distinct FTVORGN_TITLE,', ')"""
_from = f"""FORAQUS, FORAQRC, FTVORGN"""
_where = f"""FORAQUS_QUEUE_ID like '%B%'
and FORAQUS_NCHG_DATE is null
and FORAQUS_TERM_DATE is null
and replace(FORAQUS_QUEUE_ID, 'B', '') = replace(FORAQRC_QUEUE_ID, 'A', '')
and FTVORGN_ORGN_CODE = FORAQRC_ORGN_CODE"""
_order_by = f"""FORAQUS_USER_ID_APPR, FORAQUS_QUEUE_ID """
_group_by = f"""FORAQUS_QUEUE_ID, FORAQUS_USER_ID_APPR"""
_max_records_returned = f""""""
x = db_query(cur,_select,_from,_where,_order_by,_group_by,_max_records_returned)
for y in x:
    print(y)

In [ ]:
approver_un = 'SMILLER66'
_select = f"""foraqus.FORAQUS_QUEUE_ID,foraqus.FORAQUS_USER_ID_APPR,
(select count(FORAQUS_USER_ID_APPR)
from foraqus fa
where fa.FORAQUS_USER_ID_APPR != '{approver_un}'
      and fa.FORAQUS_QUEUE_ID = foraqus.FORAQUS_QUEUE_ID
                                and fa.FORAQUS_TERM_DATE is null
                                and fa.FORAQUS_NCHG_DATE is null) as OtherApprovers"""
_from = f"""foraqus"""
_where = f"""foraqus.FORAQUS_USER_ID_APPR = '{approver_un}'
and foraqus.FORAQUS_TERM_DATE is null
and foraqus.FORAQUS_NCHG_DATE is null"""
_order_by = """"""
_group_by = """"""
_max_records_returned = """"""

x = db_query(cur,_select,_from,_where,_order_by,_group_by,_max_records_returned)
for y in x:
    print(y)